#John Asencio
## Data Integration In-Class Activity

##A. [MUST] Discussion Question
Within your group, identify two or more sources of data that might be integrated to analyze a problem that could not be easily analyzed or solved otherwise. The data sources and problems do not need to be related to anything we have done in class and do not even need to be serious. Be imaginative; you do not need to describe how to integrate the data sets.

For example, you might say something like, “Integrate historic weather/precipitation data with crop yield data to help develop a system for anticipating future prices of corn and wheat.”

<insert your own ideas here and discuss them with your work group>


John - Integrating the data of position of the Earth and Mars in our Solar systema and the rotation of the Earth in order to determine the best time to launch from Earth to Mars

Trae - With the growing trend of indie/low budget video game studios that are being bought up by larger companies, there has been a diminishing amount of games being produced overall. I’d be interested in the data that showed the production of video games versus the sort of “main hobby” or interest people have in their free times.

##B. [MUST] Transform the ACS Data

The ACS data is separated into “Census Tracts” which are regions within counties that correspond to groups of approximately 4000 people. The Census Bureau defines these to help organize the actual job of collecting census data. I.e., they did it this way for their convenience, not yours, and this grouping makes your Data Engineering job more challenging. Your “dimension of overlap” is the county not the census tract, so you must transform the ACS data.

To properly integrate the ACS with the COVID data, aggregate the per-tract data to the county level of resolution.  

Create a python+pandas program that transforms the ACS data into a one-row-per-county ACS DataFrame called County_info. To do this you will need to think about how to properly aggregate Census Tract-level data into County-level summaries. Your transformation code should also eliminate unneeded columns from the ACS data.

Also, add an ID column to your County_info dataframe so that the county can be referred to by the COVID data.

In [2]:
import pandas as pd
acs_data = pd.read_csv('/content/acs2017_census_tract_data.csv.gz')

county_info = acs_data.groupby(['County', 'State']).agg({
    'TotalPop': 'sum',
    'IncomePerCap': 'mean',
    'Poverty': 'mean',
})

county_info.reset_index(inplace=True)
county_info['County_ID'] = range(1, len(county_info) + 1)

county_info

,County,State,TotalPop,IncomePerCap,Poverty,County_ID
0,Abbeville County,South Carolina,24788,19402.833333,22.183333,1
1,Acadia Parish,Louisiana,62607,21454.250000,21.933333,2
2,Accomack County,Virginia,32840,24769.750000,20.112500,3
3,Ada County,Idaho,435117,32672.661017,12.608475,4
4,Adair County,Iowa,7192,28840.666667,9.566667,5
...,...,...,...,...,...,...
3215,Yuma County,Arizona,204281,21479.641509,19.357692,3216
3216,Yuma County,Colorado,10109,24841.000000,14.350000,3217
3217,Zapata County,Texas,14415,17884.333333,32.566667,3218
3218,Zavala County,Texas,12152,12660.250000,41.375000,3219


In [30]:
#printing out data from rows to fill out table

#Loudon County info
print(county_info[(county_info['County'] == 'Loudoun County') & (county_info['State'] == 'Virginia')],'\n')

#Washington County info
print(county_info[(county_info['County'] == 'Washington County') & (county_info['State'] == 'Oregon')],'\n')

#Harlan County info
print(county_info[(county_info['County'] == 'Harlan County') & (county_info['State'] == 'Kentucky')],'\n')

#Malheur County info
print(county_info[(county_info['County'] == 'Malheur County') & (county_info['State'] == 'Oregon')],'\n')

              County     State  TotalPop  IncomePerCap   Poverty  County_ID
1759  Loudoun County  Virginia    374558  50391.015625  3.884375       1760 

                 County   State  TotalPop  IncomePerCap    Poverty  County_ID
3046  Washington County  Oregon    572071  34970.817308  10.446154       3047 

             County     State  TotalPop  IncomePerCap    Poverty  County_ID
1234  Harlan County  Kentucky     27548  16010.363636  33.318182       1235 

              County   State  TotalPop  IncomePerCap    Poverty  County_ID
1822  Malheur County  Oregon     30421  17966.428571  24.414286       1823 



Answer the following questions:
Most populous county in the USA?
Least populous county in the USA?


In [3]:
min_index = county_info['TotalPop'].idxmin()
max_index = county_info['TotalPop'].idxmax()
print(county_info.loc[min_index],'\n')
print(county_info.loc[max_index])


County          Loving County
State                   Texas
TotalPop                   74
IncomePerCap          35530.0
Poverty                  17.1
County_ID                1765
Name: 1764, dtype: object 

County          Los Angeles County
State                   California
TotalPop                  10105722
IncomePerCap          31389.413867
Poverty                  17.323803
County_ID                     1758
Name: 1757, dtype: object


##C. [MUST] Transform the COVID Data
Simplify the COVID data along the time dimension. The COVID data set contains day-level resolution data from (approximately) January of 2020 through February of 2021. From this you should produce a COVID_monthly Data Frame that has just one row per month per county.

Also, add a county ID column that is a foreign key lookup to the corresponding ID column in the County_info DataFrame.

Fill the following table using data from your COVID_monthly DataFrame:


In [18]:
covid_data = pd.read_csv('/content/COVID_county_data.csv.gz')

covid_data['date'] = pd.to_datetime(covid_data['date'])
covid_data['month'] = covid_data['date'].dt.month
covid_data['year'] = covid_data['date'].dt.year


In [78]:
covid_monthly = covid_data.groupby(['county', 'state', 'year', 'month']).agg({
    'cases': 'sum',
    'deaths': 'sum',
}).reset_index()


county_state_to_id = {(row['County'], row['State']): row['County_ID'] for index, row in county_info.iterrows()}
covid_monthly['County_ID'] = [county_state_to_id.get((county, state), None) for county, state, _, _ in zip(covid_monthly['county'], covid_monthly['state'], covid_monthly['year'], covid_monthly['month'])]

print(covid_monthly['County_ID'].isnull().sum())
covid_monthly
#for county, state in zip(covid_monthly['county'], covid_monthly['state']):
#  if county == 'Malheur':
#    print(county, state)
#    print(covid_monthly['county'] == county)
print(covid_monthly[(covid_monthly['county'] == 'Malheur') & (covid_monthly['state'] == 'Oregon')],'\n')

2048
        county   state  year  month  cases  deaths  County_ID
20777  Malheur  Oregon  2020      3      1     0.0     1823.0
20778  Malheur  Oregon  2020      4    125     0.0     1823.0
20779  Malheur  Oregon  2020      5    626     0.0     1823.0
20780  Malheur  Oregon  2020      6   1447    24.0     1823.0
20781  Malheur  Oregon  2020      7  12773   130.0     1823.0
20782  Malheur  Oregon  2020      8  28163   459.0     1823.0
20783  Malheur  Oregon  2020      9  43150   693.0     1823.0
20784  Malheur  Oregon  2020     10  56398  1061.0     1823.0
20785  Malheur  Oregon  2020     11  65787  1174.0     1823.0
20786  Malheur  Oregon  2020     12  82916  1465.0     1823.0
20787  Malheur  Oregon  2021      1  96297  1627.0     1823.0
20788  Malheur  Oregon  2021      2  65951  1137.0     1823.0 



##D. [MUST] Integrate COVID Data with ACS Data

Create a new single pandas DataFrame called COVID_summary containing one row per county. It should have these columns:
ID - integer identifier for the county
Population, Poverty, PerCapitaIncome - these should all be the same as from the County_info DataFrame
TotalCases, TotalDeaths - these two values should come from the COVID_monthly data and summed over all months
TotalCasesPer100K, TotalDeathsPer100K - these two values should be computed by dividing the the TotalCases and TotalDeaths (respectively) by (Population / 100000)



In [87]:
covid_summary = covid_monthly.groupby(['County_ID']).agg({
    'cases': 'sum',
    'deaths': 'sum'
}).reset_index()

county_info['County'] = county_info['County'].str.replace(' County', '')

#renaming so that the columns match up
covid_summary = covid_summary.rename(columns={'county': 'County', 'state':'State','cases': 'TotalCases', 'deaths': 'TotalDeaths'})

#making a list for the mismatched counties so we can see which ones will be left out
#mismatched_counties = set(county_info['County']) - set(covid_summary['County'])

#merging county_info with covid_summary based on County
covid_summary = pd.merge(county_info, covid_summary, on=['County_ID'], how='outer')

#calculating TotalCasesPer100K and TotalDeathsPer100K
covid_summary['TotalCasesPer100K'] = covid_summary['TotalCases'] / (covid_summary['TotalPop'] / 100000)
covid_summary['TotalDeathsPer100K'] = covid_summary['TotalDeaths'] / (covid_summary['TotalPop'] / 100000)


#rename ID column to match requirements
covid_summary.rename(columns={'County_ID': 'ID'}, inplace=True)
covid_summary

#for county in covid_summary['County']:
#  print(county)

,County,State,TotalPop,IncomePerCap,Poverty,ID,TotalCases,TotalDeaths,TotalCasesPer100K,TotalDeathsPer100K
0,Abbeville,South Carolina,24788,19402.833333,22.183333,1,206727.0,3934.0,8.339802e+05,15870.582540
1,Acadia Parish,Louisiana,62607,21454.250000,21.933333,2,NaN,NaN,NaN,NaN
2,Accomack,Virginia,32840,24769.750000,20.112500,3,389637.0,5758.0,1.186471e+06,17533.495737
3,Ada,Idaho,435117,32672.661017,12.608475,4,5230256.0,50724.0,1.202034e+06,11657.554175
4,Adair,Iowa,7192,28840.666667,9.566667,5,70396.0,1792.0,9.788098e+05,24916.573971
...,...,...,...,...,...,...,...,...,...,...
3215,Yuma,Arizona,204281,21479.641509,19.357692,3216,4445946.0,96197.0,2.176387e+06,47090.527264
3216,Yuma,Colorado,10109,24841.000000,14.350000,3217,68447.0,1016.0,6.770897e+05,10050.450094
3217,Zapata,Texas,14415,17884.333333,32.566667,3218,139632.0,2031.0,9.686576e+05,14089.490114
3218,Zavala,Texas,12152,12660.250000,41.375000,3219,153660.0,4211.0,1.264483e+06,34652.732061


Fill the following table using data from your COVID_summary DataFrame:


In [88]:
# Loudoun County info
print(covid_summary[(covid_summary['County'] == 'Loudoun') & (covid_summary['State'] == 'Virginia')],'\n')

# Washington County info
print(covid_summary[(covid_summary['County'] == 'Washington') & (covid_summary['State'] == 'Oregon')],'\n')

# Harlan County info
print(covid_summary[(covid_summary['County'] == 'Harlan') & (covid_summary['State'] == 'Kentucky')],'\n')

# Malheur County info
print(covid_summary[(covid_summary['County'] == 'Malheur') & (covid_summary['State'] == 'Oregon')],'\n')


       County     State  TotalPop  IncomePerCap   Poverty    ID  TotalCases  \
1759  Loudoun  Virginia    374558  50391.015625  3.884375  1760   2496450.0   

      TotalDeaths  TotalCasesPer100K  TotalDeathsPer100K  
1759      35820.0       666505.58792          9563.27191   

          County   State  TotalPop  IncomePerCap    Poverty    ID  TotalCases  \
3046  Washington  Oregon    572071  34970.817308  10.446154  3047   2157339.0   

      TotalDeaths  TotalCasesPer100K  TotalDeathsPer100K  
3046      22455.0      377110.358679          3925.21208   

      County     State  TotalPop  IncomePerCap    Poverty    ID  TotalCases  \
1234  Harlan  Kentucky     27548  16010.363636  33.318182  1235    205984.0   

      TotalDeaths  TotalCasesPer100K  TotalDeathsPer100K  
1234       3994.0       747727.60273        14498.330187   

       County   State  TotalPop  IncomePerCap    Poverty    ID  TotalCases  \
1822  Malheur  Oregon     30421  17966.428571  24.414286  1823    453634.0   

  